In [39]:
import pandas as pd
df = pd.read_csv("training.csv")
df.head()

,text,label
0,i didnt feel humiliated,0
1,i can go from feeling so hopeless to so damned...,0
2,im grabbing a minute to post i feel greedy wrong,3
3,i am ever feeling nostalgic about the fireplac...,2
4,i am feeling grouchy,3


In [40]:
df.shape

(16000, 2)

In [ ]:
texts = df['text']

0                                  i didnt feel humiliated
1        i can go from feeling so hopeless to so damned...
2         im grabbing a minute to post i feel greedy wrong
3        i am ever feeling nostalgic about the fireplac...
4                                     i am feeling grouchy
                               ...                        
15995    i just had a very brief time in the beanbag an...
15996    i am now turning and i feel pathetic that i am...
15997                       i feel strong and good overall
15998    i feel like this was such a rude comment and i...
15999    i know a lot but i feel so stupid because i ca...
Name: text, Length: 16000, dtype: object

In [42]:
texts.dtype

dtype('O')

In [43]:
texts = texts.astype('string')
texts.dtype

string[python]

In [ ]:
texts =texts.tolist()
#texts

['i didnt feel humiliated',
 'i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awake',
 'im grabbing a minute to post i feel greedy wrong',
 'i am ever feeling nostalgic about the fireplace i will know that it is still on the property',
 'i am feeling grouchy',
 'ive been feeling a little burdened lately wasnt sure why that was',
 'ive been taking or milligrams or times recommended amount and ive fallen asleep a lot faster but i also feel like so funny',
 'i feel as confused about life as a teenager or as jaded as a year old man',
 'i have been with petronas for years i feel that petronas has performed well and made a huge profit',
 'i feel romantic too',
 'i feel like i have to make the suffering i m seeing mean something',
 'i do feel that running is a divine experience and that i can expect to have some type of spiritual encounter',
 'i think it s the easiest time of year to feel dissatisfied',
 'i feel low energy i m just thirst

In [ ]:
from transformers import pipeline

emotion_model = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", top_k=None)

emotion_vectors = []
for text in texts:
    emotions = emotion_model(text)
    scores = {e["label"]: e["score"] for e in emotions[0]}
    emotion_vectors.append(scores)

In [ ]:
#emotion_vectors

[{'sadness': 0.9922937154769897,
  'disgust': 0.002003141911700368,
  'anger': 0.001669354853220284,
  'neutral': 0.0013276297831907868,
  'joy': 0.0012993129203096032,
  'surprise': 0.0009526850190013647,
  'fear': 0.0004541437956504524},
 {'sadness': 0.9922439455986023,
  'joy': 0.0031311241909861565,
  'fear': 0.0016226647421717644,
  'anger': 0.0011362340301275253,
  'neutral': 0.0008511886699125171,
  'surprise': 0.000623837229795754,
  'disgust': 0.0003909436345566064},
 {'anger': 0.9935709238052368,
  'fear': 0.0021726132836192846,
  'sadness': 0.0011588993947952986,
  'joy': 0.001092978985980153,
  'disgust': 0.0008488223538734019,
  'surprise': 0.0007187730516307056,
  'neutral': 0.0004369406960904598},
 {'joy': 0.762344241142273,
  'sadness': 0.2162228375673294,
  'surprise': 0.00893599633127451,
  'neutral': 0.004859322682023048,
  'anger': 0.003835632698610425,
  'fear': 0.002693834947422147,
  'disgust': 0.001108144293539226},
 {'anger': 0.9954922199249268,
  'sadness': 0.

In [48]:
emotion_df = pd.DataFrame(emotion_vectors)
emotion_df.head()

,sadness,disgust,anger,neutral,joy,surprise,fear
0,0.992294,0.002003,0.001669,0.001328,0.001299,0.000953,0.000454
1,0.992244,0.000391,0.001136,0.000851,0.003131,0.000624,0.001623
2,0.001159,0.000849,0.993571,0.000437,0.001093,0.000719,0.002173
3,0.216223,0.001108,0.003836,0.004859,0.762344,0.008936,0.002694
4,0.001480,0.000546,0.995492,0.000360,0.000607,0.000455,0.001060


In [49]:
df.label.value_counts()

label
1    5362
0    4666
3    2159
4    1937
2    1304
5     572
Name: count, dtype: int64

In [50]:

from sklearn.cluster import KMeans

kmeans_init = 'k-means++'

num_clusters = 6
kmeans = KMeans(n_clusters=num_clusters, init=kmeans_init, random_state=42, n_init=10)
df['Cluster'] = kmeans.fit_predict(emotion_df)

df.head(10)


,text,label,Cluster
0,i didnt feel humiliated,0,3
1,i can go from feeling so hopeless to so damned...,0,3
2,im grabbing a minute to post i feel greedy wrong,3,1
3,i am ever feeling nostalgic about the fireplac...,2,2
4,i am feeling grouchy,3,1
5,ive been feeling a little burdened lately wasn...,0,3
6,ive been taking or milligrams or times recomme...,5,0
7,i feel as confused about life as a teenager or...,4,4
8,i have been with petronas for years i feel tha...,1,2
9,i feel romantic too,2,2


In [51]:
df.Cluster.unique()

array([3, 1, 2, 0, 4, 5], dtype=int32)

In [52]:
from sklearn.metrics import classification_report

df['label'] = df['label'].astype('string') 
df.label.dtype

string[python]

In [53]:
mapping = {}
for cluster in df['Cluster'].unique():
    most_common_label = df[df['Cluster'] == cluster]['label'].mode()[0]
    mapping[cluster] = most_common_label
    
df['Predicted_Label'] = df['Cluster'].map(mapping)
df.head(20)   

,text,label,Cluster,Predicted_Label
0,i didnt feel humiliated,0,3,0
1,i can go from feeling so hopeless to so damned...,0,3,0
2,im grabbing a minute to post i feel greedy wrong,3,1,3
3,i am ever feeling nostalgic about the fireplac...,2,2,1
4,i am feeling grouchy,3,1,3
5,ive been feeling a little burdened lately wasn...,0,3,0
6,ive been taking or milligrams or times recomme...,5,0,5
7,i feel as confused about life as a teenager or...,4,4,4
8,i have been with petronas for years i feel tha...,1,2,1
9,i feel romantic too,2,2,1


In [54]:
df.Predicted_Label.unique()

array(['0', '3', '1', '5', '4'], dtype=object)

In [55]:
mapping

{3: '0', 1: '3', 2: '1', 0: '5', 4: '4', 5: '0'}

In [56]:
print("Classification Report:")
print(classification_report(df['label'], df['Predicted_Label']))


Classification Report:


/home/mntel/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

           0       0.89      0.93      0.91      4666
           1       0.83      0.93      0.87      5362
           2       0.00      0.00      0.00      1304
           3       0.87      0.92      0.90      2159
           4       0.87      0.91      0.89      1937
           5       0.71      0.93      0.80       572

    accuracy                           0.85     16000
   macro avg       0.69      0.77      0.73     16000
weighted avg       0.78      0.85      0.81     16000



/home/mntel/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/mntel/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
